# **Library and data import**

In [3]:
# import of python libraries
from google.colab import files # for file upload to the google colab server
import pandas as pd # data import of CV-files
import numpy as np # for numerical operations
import matplotlib.pyplot as plt # for plotting of data
from scipy.optimize import curve_fit # curve fitting tool

In [ ]:
details = files.upload() # google colab file upload query
filename=list(details.keys())[0]

In [ ]:
usecols = ["I_DD", "I_DA", "I_AA", "BG_DD", "BG_DA", "BG_AA"] # selected columns of CSV-file for data import
data = pd.read_csv(filename,sep=",",usecols=usecols) # read data from CSV-file
data.head() # table preview

# **Calculation of corrected FRET efficiency (E) and corrected fluorescence stoichiometry (S)**

In [6]:
# Intensity values
I_DD=data.values[:,0] # donor signal after donor excitation
I_DA=data.values[:,1] # acceptor signal after donor excitation (FRET)
I_AA=data.values[:,2] # acceptor signal after acceptor excitation

# Corresponding background values
BG_DD=data.values[:,3]
BG_DA=data.values[:,4]
BG_AA=data.values[:,5]

In [7]:
# Background correction
# !!!PLEASE COMPLETE THE EQUATIONS FOR THE BACKGROUND CORRECTED INTENSITIES!!!
F_DD=
F_DA=
F_AA=

In [8]:
# Uncorrected data
# !!!PLEASE COMPLETE THE EQUATIONS FOR THE UNCORRECTED FRET EFFICIENCY AND STOICHIOMETRY!!!
E = # apparent FRET efficiency
S = # apparent fluorescence stoichiometry

In [ ]:
# data plotting
plt.figure()
plt.plot(E, S, '.', color='grey', markersize=3)
plt.xlabel('FRET efficiency, $\it{E}_{\mathrm{app}}$')
plt.ylabel('Stoichiometry, $\it{S}_{\mathrm{app}}$')
plt.axis([-0.1, 1.1, -0.1, 1.1])
plt.title("Uncorrected data")

In [10]:
# Corrected data
# Correction factors
# !!!PLEASE FILL IN THE CORRECTION FACTORS FOR CY3 AND CY5!!!
alpha= # correction for direct excitation of the acceptor by the donor laser
beta= # correction for spectral crosstalk of the donor signal in the acceptor detection channel
gamma= # correction for differences in quantum yield and detection efficiency

# !!!PLEASE COMPLETE THE EQUATIONS FOR THE CORRECTED FRET EFFICIENCY AND STOICHIOMETRY!!!
Ecorr =  # corrected FRET efficiency
Scorr =  # corrected stoichiometry

In [ ]:
# data plotting
plt.figure()
plt.plot(Ecorr, Scorr, '.', color='grey', markersize=3)
plt.xlabel('FRET efficiency, $\it{E}_{\mathrm{corr}}$')
plt.ylabel('Stoichiometry, $\it{S}_{\mathrm{corr}}$')
plt.axis([-0.1, 1.1, -0.1, 1.1])
plt.title("Corrected data")

# **Extraction of double labeled molecules for FRET analysis**

In [12]:
# stoichiometry thresholds to extract molecules with donor and acceptor active S=0.5
# !!!PLEASE CHOOSE THRESHOLDS!!!
min_S =
max_S =

# stoichiometry filtered bursts
sel_Ecorr=Ecorr[np.logical_and(Scorr>min_S, Scorr<max_S)]
sel_Scorr=Scorr[np.logical_and(Scorr>min_S, Scorr<max_S)]

In [ ]:
# data plotting
plt.figure()
plt.plot(Ecorr, Scorr, '.', color='grey', markersize=3)
plt.plot(sel_Ecorr, sel_Scorr, '+r')
plt.xlabel('FRET efficiency, $\it{E}_{\mathrm{corr}}$')
plt.ylabel('Stoichiometry, $\it{S}_{\mathrm{corr}}$')
plt.axis([-0.1, 1.1, -0.1, 1.1])
plt.title("Corrected and filtered data")

# **Sorting of filtered molecules into an E-histogram**

In [14]:
# calculation of FRET efficiency histogram of filtered data
bin_width=0.025 # bin width of histogram bars
edges=np.arange(-0.1+0.45*bin_width, 1.1, bin_width) # bin edges

# FRET efficiency histogram
hist_E, hist_E_edges = np.histogram(sel_Ecorr, edges)
x_values=edges[0:-1]+0.5*bin_width

In [ ]:
# Histogram plotting
plt.figure()
plt.hist(sel_Ecorr, bins=edges)
plt.xlabel('FRET efficiency, $\it{E}_{\mathrm{corr}}$')
plt.ylabel('# Molecules')
plt.xlim(-0.1, 1.1)

# **Modeling of FRET efficiency data**

In [16]:
# Fitting of histogram data
def gauss_function(x, A, x0, sig): # single Gaussian distribution
    return A*np.exp(-(x-x0)**2/(2*sig**2))
def gauss2_function(x, A1, x0_1, sig1, A2, x0_2, sig2): # two Gaussian distributions
    return A1*np.exp(-(x-x0_1)**2/(2*sig1**2))+A2*np.exp(-(x-x0_2)**2/(2*sig2**2))

# !!!PLEASE COMPLETE THE FUNCTION FOR THREE GAUSSIAN DISTRIBUTIONS!!!
def gauss3_function(x, A1, x0_1, sig1, A2, x0_2, sig2, A3, x0_3, sig3): # three Gaussian distributions
    return

**Single Gaussian distribution**

In [ ]:
# Fit of single Gaussian distribution
ini_A = 2000
ini_x0 = 0
ini_sig = 0.1

# curve fitting
fit_parameters, covariance = curve_fit(gauss_function, x_values, hist_E, p0 = [ini_A, ini_x0, ini_sig], bounds=(0, [np.inf, 1, 0.2]))

# extraction of fitting parameters
optimal_A = fit_parameters[0]
optimal_x0 = fit_parameters[1]
optimal_sig = fit_parameters[2]
error_A = np.sqrt(covariance[0,0])
error_x0 = np.sqrt(covariance[1,1])
error_sig = np.sqrt(covariance[2,2])

# fitting curve
fitG = gauss_function(x_values, optimal_A, optimal_x0, optimal_sig)

# fit residual
fit_residual=(hist_E - fitG)
reduced_chi_square=np.sum(((hist_E[hist_E!=0] - fitG[hist_E!=0])**2)/hist_E[hist_E!=0])/(np.sum([hist_E!=0])-3)

# Plot of the fit result
plt.figure()
plt.subplot2grid((3, 1), (0, 0), colspan=1, rowspan=2)
plt.hist(sel_Ecorr, bins=edges)
plt.plot(x_values, gauss_function(x_values, optimal_A, optimal_x0, optimal_sig), '-r')
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
plt.ylabel('# Molecules')
plt.xlim(-0.1, 1.1)
plt.subplot2grid((3, 1), (2, 0), colspan=1, rowspan=1)
plt.plot(x_values, fit_residual, '-k')
plt.xlabel('FRET efficiency, $\it{E}_{\mathrm{corr}}$')
plt.ylabel('Residual')
plt.xlim(-0.1, 1.1)

plt.show()

# display fitting parameters
print("Gauss \n-------------------")
print("A= {a1: 4.3f} +-{a2: 4.3f}".format(a1 = optimal_A, a2 = error_A))
print("x0= {b1: 4.3f} +-{b2: 4.3f}".format(b1 = optimal_x0, b2 = error_x0))
print("sig= {c1: 4.3f} +-{c2: 4.3f}".format(c1 = optimal_sig, c2 = error_sig))
print("\nRed. chi-squared= {k: 4.3f}".format(k= reduced_chi_square))

**Double Gaussian distribution**

In [ ]:
# Fit of two Gaussian distributions one fixed at E=0
ini_A1 = 1000
ini_x0_1 = 0
ini_sig1 = 0.1
ini_A2 = 1000
ini_x0_2 = 0.2
ini_sig2 = 0.1

# curve fitting
fit_parameters, covariance = curve_fit(gauss2_function, x_values, hist_E, p0 = [ini_A1, ini_x0_1, ini_sig1, ini_A2, ini_x0_2, ini_sig2], bounds=(0, [np.inf, 0.01, 0.2, np.inf, 1, 0.2]))

# extraction of fitting parameters
optimal_A1 = fit_parameters[0]
optimal_x0_1 = fit_parameters[1]
optimal_sig1 = fit_parameters[2]
optimal_A2 = fit_parameters[3]
optimal_x0_2 = fit_parameters[4]
optimal_sig2 = fit_parameters[5]
error_A1 = np.sqrt(covariance[0,0])
error_x0_1 = np.sqrt(covariance[1,1])
error_sig1 = np.sqrt(covariance[2,2])
error_A2 = np.sqrt(covariance[3,3])
error_x0_2 = np.sqrt(covariance[4,4])
error_sig2 = np.sqrt(covariance[5,5])

# individual fitting curves
fitG1 = gauss_function(x_values, optimal_A1, optimal_x0_1, optimal_sig1)
fitG2 = gauss_function(x_values, optimal_A2, optimal_x0_2, optimal_sig2)
fitG12 = gauss2_function(x_values, optimal_A1, optimal_x0_1, optimal_sig1, optimal_A2, optimal_x0_2, optimal_sig2)

# fraction of molecules of each population and their error
frac1=np.sum(fitG1)/np.sum(fitG12)
frac2=np.sum(fitG2)/np.sum(fitG12)
err_frac1=np.sqrt(frac1+frac1**2)/np.sqrt(np.sum(fitG12))
err_frac2=np.sqrt(frac2+frac2**2)/np.sqrt(np.sum(fitG12))

# fit residual
fit_residual=(hist_E - fitG12)
reduced_chi_square=np.sum(((hist_E[hist_E!=0] - fitG12[hist_E!=0])**2)/hist_E[hist_E!=0])/(np.sum([hist_E!=0])-5)

# Plot of the fit result
plt.figure()
plt.subplot2grid((3, 1), (0, 0), colspan=1, rowspan=2)
plt.hist(sel_Ecorr, bins=edges)
plt.plot(x_values, fitG1, '--k')
plt.plot(x_values, fitG2, '--k')
plt.plot(x_values, fitG12, '-r')
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
plt.ylabel('# Molecules')
plt.xlim(-0.1, 1.1)
plt.subplot2grid((3, 1), (2, 0), colspan=1, rowspan=1)
plt.plot(x_values, fit_residual, '-k')
plt.xlabel('FRET efficiency, $\it{E}_{\mathrm{corr}}$')
plt.ylabel('Residual')
plt.xlim(-0.1, 1.1)

plt.show()

# display fitting parameters
print("Gauss 1\n-------------------")
print("A= {a1: 4.3f} +-{a2: 4.3f}".format(a1 = optimal_A1, a2 = error_A1))
print("x0= {b1: 4.3f} +-{b2: 4.3f}".format(b1 = optimal_x0_1, b2 = error_x0_1))
print("sig= {c1: 4.3f} +-{c2: 4.3f}".format(c1 = optimal_sig1, c2 = error_sig1))
print("\nGauss 2\n-------------------")
print("A= {a1: 4.3f} +-{a2: 4.3f}".format(a1 = optimal_A2, a2 = error_A2))
print("x0= {b1: 4.3f} +-{b2: 4.3f}".format(b1 = optimal_x0_2, b2 = error_x0_2))
print("sig= {c1: 4.3f} +-{c2: 4.3f}".format(c1 = optimal_sig2, c2 = error_sig2))
print("\nFraction of molecules in populations \n-------------------")
print("frac1= {l1: 4.3f} +-{l2: 4.3f}".format(l1= frac1, l2=err_frac1))
print("frac2= {l1: 4.3f} +-{l2: 4.3f}".format(l1= frac2, l2=err_frac2))
print("\nRed. chi-squared= {k: 4.3f}".format(k= reduced_chi_square))

**Three Gaussian distributions**

In [ ]:
# Fit of three Gaussian distributions one fixed at E=0
ini_A1 = 1000
ini_sig1 = 0.1
ini_x0_1 = 0
ini_A2 = 1000
ini_x0_2 = 0.2
ini_sig2 = 0.1
ini_A3 = 1000
ini_x0_3 = 0.5
ini_sig3 = 0.1

# curve fitting
fit_parameters, covariance = curve_fit(gauss3_function, x_values, hist_E, p0 = [ini_A1, ini_x0_1, ini_sig1, ini_A2, ini_x0_2, ini_sig2, ini_A3, ini_x0_3, ini_sig3], bounds=(0, [np.inf, 0.01, 0.2, np.inf, 1, 0.2, np.inf, 1, 0.2]))

# extraction of fitting parameters
optimal_A1 = fit_parameters[0]
optimal_x0_1 = fit_parameters[1]
optimal_sig1 = fit_parameters[2]
optimal_A2 = fit_parameters[3]
optimal_x0_2 = fit_parameters[4]
optimal_sig2 = fit_parameters[5]
optimal_A3 = fit_parameters[6]
optimal_x0_3 = fit_parameters[7]
optimal_sig3 = fit_parameters[8]
error_A1 = np.sqrt(covariance[0,0])
error_x0_1 = np.sqrt(covariance[1,1])
error_sig1 = np.sqrt(covariance[2,2])
error_A2 = np.sqrt(covariance[3,3])
error_x0_2 = np.sqrt(covariance[4,4])
error_sig2 = np.sqrt(covariance[5,5])
error_A3 = np.sqrt(covariance[6,6])
error_x0_3 = np.sqrt(covariance[7,7])
error_sig3 = np.sqrt(covariance[8,8])

# individual fitting curves
fitG1 = gauss_function(x_values, optimal_A1, optimal_x0_1, optimal_sig1)
fitG2 = gauss_function(x_values, optimal_A2, optimal_x0_2, optimal_sig2)
fitG3 = gauss_function(x_values, optimal_A3, optimal_x0_3, optimal_sig3)
fitG123 = gauss3_function(x_values, optimal_A1, optimal_x0_1, optimal_sig1, optimal_A2, optimal_x0_2, optimal_sig2, optimal_A3, optimal_x0_3, optimal_sig3)

# fraction of molecules of each population and their error
frac1=np.sum(fitG1)/np.sum(fitG123)
frac2=np.sum(fitG2)/np.sum(fitG123)
frac3=np.sum(fitG3)/np.sum(fitG123)
err_frac1=np.sqrt(frac1+frac1**2)/np.sqrt(np.sum(fitG123))
err_frac2=np.sqrt(frac2+frac2**2)/np.sqrt(np.sum(fitG123))
err_frac3=np.sqrt(frac3+frac3**2)/np.sqrt(np.sum(fitG123))

# fit residual
fit_residual=(hist_E - fitG123)
reduced_chi_square=np.sum(((hist_E[hist_E!=0] - fitG123[hist_E!=0])**2)/hist_E[hist_E!=0])/(np.sum([hist_E!=0])-8)

# Plot of the fit result
plt.figure()
plt.subplot2grid((3, 1), (0, 0), colspan=1, rowspan=2)
plt.hist(sel_Ecorr, bins=edges)
plt.plot(x_values, fitG1, '--k')
plt.plot(x_values, fitG2, '--k')
plt.plot(x_values, fitG3, '--k')
plt.plot(x_values, fitG123, '-r')
frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
plt.ylabel('# Molecules')
plt.xlim(-0.1, 1.1)
plt.subplot2grid((3, 1), (2, 0), colspan=1, rowspan=1)
plt.plot(x_values, fit_residual, '-k')
plt.xlabel('FRET efficiency, $\it{E}_{\mathrm{corr}}$')
plt.ylabel('Residual')
plt.xlim(-0.1, 1.1)

plt.show()

# display fitting parameters
print("Gauss 1\n-------------------")
print("A= {a1: 4.3f} +-{a2: 4.3f}".format(a1 = optimal_A1, a2 = error_A1))
print("x0= {b1: 4.3f} +-{b2: 4.3f}".format(b1 = optimal_x0_1, b2 = error_x0_1))
print("sig= {c1: 4.3f} +-{c2: 4.3f}".format(c1 = optimal_sig1, c2 = error_sig1))
print("\nGauss 2\n-------------------")
print("A= {a1: 4.3f} +-{a2: 4.3f}".format(a1 = optimal_A2, a2 = error_A2))
print("x0= {b1: 4.3f} +-{b2: 4.3f}".format(b1 = optimal_x0_2, b2 = error_x0_2))
print("sig= {c1: 4.3f} +-{c2: 4.3f}".format(c1 = optimal_sig2, c2 = error_sig2))
print("\nGauss 3\n-------------------")
print("A= {a1: 4.3f} +-{a2: 4.3f}".format(a1 = optimal_A3, a2 = error_A3))
print("x0= {b1: 4.3f} +-{b2: 4.3f}".format(b1 = optimal_x0_3, b2 = error_x0_3))
print("sig= {c1: 4.3f} +-{c2: 4.3f}".format(c1 = optimal_sig3, c2 = error_sig3))
print("\nFraction of molecules\n-------------------")
print("frac1= {l1: 4.3f} +-{l2: 4.3f}".format(l1= frac1, l2=err_frac1))
print("frac2= {l1: 4.3f} +-{l2: 4.3f}".format(l1= frac2, l2=err_frac2))
print("frac3= {l1: 4.3f} +-{l2: 4.3f}".format(l1= frac3, l2=err_frac3))
print("\nRed. chi-squared= {k: 4.3f}".format(k= reduced_chi_square))